<a href="https://colab.research.google.com/github/vinifborgess/aviationstack/blob/main/API_Project_AviationStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import requests
import pprint

# URL and Params
url = 'https://api.aviationstack.com/v1/flights'
params = {
  'access_key': '1666612900443c14dc413bc4df9d749b',
}

# API Request
api_result = requests.get(url, params=params)
api_response = api_result.json()

# Payload
print("API response:")
pprint.pprint(api_response)

API response:
{'data': [{'aircraft': None,
           'airline': {'iata': 'JL',
                       'icao': 'JTL',
                       'name': 'Jet Linx Aviation'},
           'arrival': {'actual': None,
                       'actual_runway': None,
                       'airport': 'Internacional Benito Juarez',
                       'baggage': None,
                       'delay': None,
                       'estimated': '2024-04-21T07:00:00+00:00',
                       'estimated_runway': None,
                       'gate': None,
                       'iata': 'MEX',
                       'icao': 'MMMX',
                       'scheduled': '2024-04-21T07:00:00+00:00',
                       'terminal': '2',
                       'timezone': 'America/Mexico_City'},
           'departure': {'actual': None,
                         'actual_runway': None,
                         'airport': 'Narita International Airport',
                         'delay': 10,
              

In [30]:
import requests
import csv
from datetime import datetime

def get_qatar_airways_flights():
    url = 'https://api.aviationstack.com/v1/flights'
    params = {
        'access_key': '1666612900443c14dc413bc4df9d749b',
        'airline_iata': 'qr',
        'airline_icao': 'qtr'
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data

def process_and_store_data(data, filename):
    if not data or 'data' not in data:
        print("No flight data available.")
        return

    flights = data['data']

    # ETL (ET)
    fieldnames = set()
    for flight in flights:
        for key, value in flight.items():
            if isinstance(value, dict):
                for sub_key in value.keys():
                    fieldnames.add(f"{key}_{sub_key}")
            else:
                fieldnames.add(key)

    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for flight in flights:
            row = {}
            for key, value in flight.items():
                if isinstance(value, dict):
                    for sub_key, sub_value in value.items():
                        row[f"{key}_{sub_key}"] = sub_value
                else:
                    row[key] = value
            writer.writerow(row)

    print(f"Flight data stored in {filename}")

# ETL (L)
filename = 'qatar_airways_flights.csv'

# ETL (E)
flight_data = get_qatar_airways_flights()

# ETL (L)
process_and_store_data(flight_data, filename)


Flight data stored in qatar_airways_flights.csv


In [31]:
import sqlite3
import requests
from datetime import datetime

def get_qatar_airways_flights():
    url = 'https://api.aviationstack.com/v1/flights'
    params = {
        'access_key': '1666612900443c14dc413bc4df9d749b',
        'airline_iata': 'qr',
        'airline_icao': 'qtr'
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data

def create_flight_table(conn):
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS flights (
                        id INTEGER PRIMARY KEY,
                        flight_date TEXT,
                        flight_number TEXT,
                        departure_airport TEXT,
                        arrival_airport TEXT,
                        departure_terminal TEXT,
                        arrival_terminal TEXT,
                        departure_gate TEXT,
                        arrival_gate TEXT,
                        departure_scheduled TEXT,
                        departure_estimated TEXT,
                        departure_actual TEXT,
                        arrival_scheduled TEXT,
                        arrival_estimated TEXT,
                        arrival_actual TEXT
                    )''')
    conn.commit()

def insert_flight_data(conn, flights):
    cursor = conn.cursor()
    for flight in flights:
        cursor.execute('''INSERT INTO flights (
                            flight_date, flight_number, departure_airport, arrival_airport,
                            departure_terminal, arrival_terminal, departure_gate, arrival_gate,
                            departure_scheduled, departure_estimated, departure_actual,
                            arrival_scheduled, arrival_estimated, arrival_actual
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                       (flight.get('flight_date'),
                        flight.get('flight_number'),
                        flight.get('departure').get('airport'),
                        flight.get('arrival').get('airport'),
                        flight.get('departure').get('terminal'),
                        flight.get('arrival').get('terminal'),
                        flight.get('departure').get('gate'),
                        flight.get('arrival').get('gate'),
                        flight.get('departure').get('scheduled'),
                        flight.get('departure').get('estimated'),
                        flight.get('departure').get('actual'),
                        flight.get('arrival').get('scheduled'),
                        flight.get('arrival').get('estimated'),
                        flight.get('arrival').get('actual')))
    conn.commit()

def main():
    # Database connect
    conn = sqlite3.connect('flights.db')

    # Creating table
    create_flight_table(conn)

    # ETL (E)
    flight_data = get_qatar_airways_flights()

    # Injecting data results
    if flight_data and 'data' in flight_data:
        insert_flight_data(conn, flight_data['data'])
        print("Flight data inserted into database.")
    else:
        print("No flight data available.")

    # Close
    conn.close()

if __name__ == "__main__":
    main()


Flight data inserted into database.


In [32]:
import sqlite3

def view_flight_data():
    conn = sqlite3.connect('flights.db')
    cursor = conn.cursor()

    cursor.execute('''SELECT * FROM flights''')
    flights = cursor.fetchall()

    conn.close()

    return flights

def main():
    flights = view_flight_data()

    if flights:
        for flight in flights:
            print(flight)
    else:
        print("No flight data available.")

if __name__ == "__main__":
    main()


(1, '2024-04-21', None, 'Doha International', 'Erbil International (irbil Northwest)', None, None, 'C32', None, '2024-04-21T00:55:00+00:00', '2024-04-21T00:55:00+00:00', None, '2024-04-21T03:20:00+00:00', '2024-04-21T03:20:00+00:00', None)
(2, '2024-04-21', None, 'Cairo International Airport', 'Doha International', '2', None, None, None, '2024-04-21T01:20:00+00:00', '2024-04-21T01:20:00+00:00', None, '2024-04-21T05:25:00+00:00', '2024-04-21T05:25:00+00:00', None)
(3, '2024-04-21', None, 'Koh Samui', 'Phuket International', None, 'D', None, None, '2024-04-21T08:00:00+00:00', '2024-04-21T08:00:00+00:00', None, '2024-04-21T09:00:00+00:00', '2024-04-21T09:00:00+00:00', None)
(4, '2024-04-21', None, 'Sultan Mahmood', 'Kuala Lumpur International Airport (klia)', None, '1', None, None, '2024-04-21T08:45:00+00:00', '2024-04-21T08:45:00+00:00', None, '2024-04-21T09:45:00+00:00', '2024-04-21T09:45:00+00:00', None)
(5, '2024-04-21', None, 'Noibai International', 'Doha International', '2', None, N